In [1]:
import pandas as pd
import re
import math
import numpy as np
import ast


In [2]:
gp_df = pd.read_csv('data/priority_claims.csv', sep='\t')

In [4]:
def clean_text(text):
    if isinstance(text, float):
        return text
    processed = text.lower()
    processed = re.sub(r'[#|\!|\-|\+|:|//|,|\"|\[|\]]', " ", processed)
    processed = re.sub(r"'", "", processed)
    processed = re.sub(r'"', "", processed)
    processed = re.sub(r'(?:(?:\d+,?)+(?:\.?\d+)?)', ' ', processed)
    processed = re.sub('[\s]+', ' ', processed)
    return processed

In [8]:
gp_df["abstract_clean"] = gp_df["abstract_text"].apply(clean_text)
gp_df["title_clean"] = gp_df["title_text"].apply(clean_text)
gp_df.to_csv("priority_claims_clean.csv",index=False, sep='\t')

In [9]:
disease_syn = pd.read_excel("mortality_data/disease_synonyms.xlsx")
disease_syn = disease_syn[["ICD_code", "entry_term"]]
# list of ICD disease groups
codes = list(disease_syn.ICD_code.unique())

In [10]:
def search_patent(df, terms, ob=False):
    
    result=[]
    for term in terms:
        clean_term = clean_text(term)
        result = result + df.loc[(~df.abstract_clean.isnull()) & (df.abstract_clean.str.contains(clean_term))].publication_number.tolist()
        result = result + df.loc[(~df.title_clean.isnull()) & (df.title_clean.str.contains(clean_term))].publication_number.tolist()
        if ob:
            result = result + df.loc[(~df.mesh_clean.isnull()) & (df.mesh_clean.str.contains(clean_term))].publication_number.tolist()

    result = list(set(result))
    return result

In [ ]:
# keyword search through google patent data
for c in codes:
    print("Searching patents for %s" % c)
    search_terms = disease_syn.loc[disease_syn.ICD_code == c].entry_term.values
    patents = search_patent(gp_df, search_terms)
    fname = 'data/result/%s.txt' % c
    with open(fname, 'w') as f:
        for p in patents:
            f.write("%s\n" % p)
    
    print("Wrote %s" % fname)
    print("...")

In [18]:
ob_df = pd.read_csv("data/ob_patents.csv",sep='\t')
ob_df["title_clean"] = ob_df["title_text"].apply(clean_text)
ob_df["abstract_clean"] = ob_df["abstract_text"].apply(clean_text)
ob_df["mesh_clean"] = ob_df["mesh_heading"].apply(clean_text)


In [12]:
# keyword search through orangebook data
for c in codes:
    print("Searching patents for %s" % c)
    search_terms = disease_syn.loc[disease_syn.ICD_code == c].entry_term.values
    patents = search_patent(ob_df, search_terms, ob=True)
    fname = 'data/ob_result/%s.txt' % c
    with open(fname, 'w') as f:
        for p in patents:
            f.write("%s\n" % p)
    
    print("Wrote %s" % fname)
    print("...")

Searching patents for A00
Wrote data/ob_result/A00.txt
...
Searching patents for A01
Wrote data/ob_result/A01.txt
...
Searching patents for A02
Wrote data/ob_result/A02.txt
...
Searching patents for A03
Wrote data/ob_result/A03.txt
...
Searching patents for A04
Wrote data/ob_result/A04.txt
...
Searching patents for A05
Wrote data/ob_result/A05.txt
...
Searching patents for A06,A07
Wrote data/ob_result/A06,A07.txt
...
Searching patents for A08
Wrote data/ob_result/A08.txt
...
Searching patents for A09
Wrote data/ob_result/A09.txt
...
Searching patents for A15,A16
Wrote data/ob_result/A15,A16.txt
...
Searching patents for A17,A18,A19
Wrote data/ob_result/A17,A18,A19.txt
...
Searching patents for A20
Wrote data/ob_result/A20.txt
...
Searching patents for A21
Wrote data/ob_result/A21.txt
...
Searching patents for A22
Wrote data/ob_result/A22.txt
...
Searching patents for A23
Wrote data/ob_result/A23.txt
...
Searching patents for A24
Wrote data/ob_result/A24.txt
...
Searching patents for A2

/Users/jaeyoonjung/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  
/Users/jaeyoonjung/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  import sys
/Users/jaeyoonjung/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if __name__ == '__main__':


Wrote data/ob_result/B01.txt
...
Searching patents for B02
Wrote data/ob_result/B02.txt
...
Searching patents for B03
Wrote data/ob_result/B03.txt
...
Searching patents for B04
Wrote data/ob_result/B04.txt
...
Searching patents for B05,B06
Wrote data/ob_result/B05,B06.txt
...
Searching patents for B07
Wrote data/ob_result/B07.txt
...
Searching patents for B08,B09
Wrote data/ob_result/B08,B09.txt
...
Searching patents for B15,B16,B17,B18,B19
Wrote data/ob_result/B15,B16,B17,B18,B19.txt
...
Searching patents for B20,B21,B22,B23,B24
Wrote data/ob_result/B20,B21,B22,B23,B24.txt
...
Searching patents for B25,B27,B30
Wrote data/ob_result/B25,B27,B30.txt
...
Searching patents for B26
Wrote data/ob_result/B26.txt
...
Searching patents for B33
Wrote data/ob_result/B33.txt
...
Searching patents for B34
Wrote data/ob_result/B34.txt
...
Searching patents for B35,B36,B37,B38,B39,B40,B41,B42,B43,B44,B45,B46,B47,B48,B49
Wrote data/ob_result/B35,B36,B37,B38,B39,B40,B41,B42,B43,B44,B45,B46,B47,B48,B49.

In [15]:
# read the list of patents that match keyword search!
matched_patents = pd.DataFrame(columns=["ICD_code", "publication_number"])
for c in codes:
    fname = 'data/result/%s.txt' % c
    with open(fname) as f:
        patents = f.read().splitlines()
    
    searched = pd.DataFrame({"ICD_code": [c]*len(patents), "publication_number":patents})
    matched_patents = matched_patents.append(searched, ignore_index=True)
    

In [16]:
for c in codes:
    fname = 'data/ob_result/%s.txt' % c
    with open(fname) as f:
        patents = f.read().splitlines()
    
    searched = pd.DataFrame({"ICD_code": [c]*len(patents), "publication_number":patents})
    matched_patents = matched_patents.append(searched, ignore_index=True)
    

In [ ]:
matched_patents = matched_patents.drop_duplicates().reset_index(drop=True)
matched_patents = matched_patents.sort_values(by="ICD_code").reset_index(drop=True)
matched_patents.to_csv("data/disease_patent_match.csv", sep='\t', index=False)

In [23]:
matched_patents = pd.read_csv("data/disease_patent_match.csv", sep="\t")

In [24]:
# get list of patents and their families
families = pd.concat([gp_df[["publication_number","family_id"]], ob_df[["publication_number","family_id"]]])
families = families.drop_duplicates()
families= families.reset_index(drop=True)

In [25]:
# find family Ids of each matched patent
matched_patents_fam = pd.merge(left=matched_patents, right=families, on="publication_number", how="left")
# find other patents in the same family 
matched_patents_fam = pd.merge(left = matched_patents_fam, right=families, on="family_id", how="left")
matched_patents_fam = matched_patents_fam.drop("publication_number_x", axis=1)
matched_patents_fam = matched_patents_fam.drop("family_id", axis=1)
matched_patents_fam = matched_patents_fam.rename(columns={"publication_number_y":"publication_number"})
matched_patents_fam = matched_patents_fam.drop_duplicates().reset_index(drop=True)

In [28]:
del(families)

In [27]:
patent_cols = ["publication_number","country_code", "publication_date", "priority_year", "filing_year",
               "assignees_harmonized", "citations", "ipc_codes"]
all_patents = pd.concat([gp_df[patent_cols], ob_df[patent_cols]])

In [29]:
all_patents = all_patents.drop_duplicates("publication_number")
all_patents = all_patents.reset_index(drop=True)
all_patents["publication_year"] = all_patents["publication_date"].apply(lambda x: int(str(x)[0:4]))

In [30]:
def count_citation(x):
    if isinstance(x, str):
        return len(x.split(','))
    else:
        return 0
all_patents["citation_count"]= all_patents["citations"].apply(count_citation)
all_patents = all_patents.drop(["publication_date"], axis=1)
matched_patents_fam=pd.merge(left = matched_patents_fam,right=all_patents, on="publication_number", how="left")

In [37]:
matched_patents_fam.to_csv("data/disease_patent_match_extended.csv", sep='\t', index=False)

In [39]:
disease_group = pd.read_excel("mortality_data/disease_code.xlsx")
disease_custom_group = disease_group[["neglected", "custom_group", 
                                      "custom_group_code"]].drop_duplicates().reset_index(drop=True)

In [40]:
matched_patents_full = pd.merge(left = matched_patents_fam, right = disease_custom_group,
        left_on="ICD_code", right_on="custom_group_code", how="left")

In [73]:
by_disease_group = matched_patents_full.groupby(["ICD_code","custom_group","neglected", "filing_year"])\
    .agg({"publication_number":"count"}).reset_index()
#by_disease_group["log_patent_count"] = by_disease_group["publication_number"].apply(math.log)
by_disease_group = by_disease_group.loc[by_disease_group.filing_year >= 1984]
by_disease_group = by_disease_group.reset_index(drop=True)
by_disease_group =  by_disease_group.rename(columns={"publication_number":"patent_count"})
by_disease_group["filing_year"] = by_disease_group["filing_year"].apply(int)

In [74]:
by_disease_group.to_csv("data/by_disease_group.csv",index=False, sep="\t")

In [75]:
# fill in missing year
by_disease_group_complete = by_disease_group.copy()

dgs = by_disease_group.ICD_code.unique()
for dg in dgs:
    dg_df = by_disease_group.loc[by_disease_group.ICD_code == dg]
    dg_start_year = max([dg_df.filing_year.min(), 1984])
    year_list = list(range(dg_start_year, 2019))
    dg_name = dg_df.custom_group.values[0]
    dg_neglected = dg_df.neglected.values[0]
    zero_row = dict({"ICD_code":dg, "custom_group":dg_name, "neglected":dg_neglected, 
                     "patent_count":0})
    for y in year_list:
        if y not in dg_df.filing_year.values:
            zero_row["filing_year"] = y
            by_disease_group_complete = by_disease_group_complete.append(zero_row, ignore_index=True)
        

In [77]:
by_disease_group.shape

(2385, 5)

In [82]:
by_disease_group_complete.head()

,ICD_code,custom_group,neglected,filing_year,patent_count
0,A00,Cholera,True,1984,4
1,A00,Cholera,True,1985,3
2,A00,Cholera,True,1986,7
3,A00,Cholera,True,1987,8
4,A00,Cholera,True,1988,9


In [84]:
by_disease_group_complete = by_disease_group_complete.sort_values(["ICD_code",  "filing_year"]).reset_index(drop=True)

In [96]:
by_disease_group_complete["log_patent_count0"] = np.log(by_disease_group_complete["patent_count"] )
by_disease_group_complete.loc[by_disease_group_complete.patent_count == 0, "log_patent_count0"] = np.NaN
by_disease_group_complete["log_patent_count1"] = np.log(by_disease_group_complete["patent_count"] + 1)

/Users/jaeyoonjung/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


In [100]:
by_disease_group_complete.to_csv("data/by_disease_group_complete.csv", sep='\t', index=False)

## get mortality data

In [758]:
mort = pd.read_csv("data/mortality_data.csv", sep="\t")
mort = pd.merge(left=mort, right=disease_group[["ICD_code", "custom_group_code"]], 
                left_on="cause", right_on="ICD_code")
mort = mort.groupby(["Year", "custom_group_code", "country"]).Deaths1.sum().reset_index()

In [759]:
mort = mort.rename(columns={"custom_group_code":"ICD_code", "Deaths1":"deaths", "Year":"year"})

In [760]:
mort.loc[mort.country=="United States of America", "country"] = "United States"

In [761]:
# fill in missing year
mort_complete = mort.copy()
#mort_complete = pd.DataFrame(columns=mort.columns)
mort_countries = mort.country.unique()
dgs = mort.ICD_code.unique()
count = 0
for mc in mort_countries:
    mc_df = mort.loc[mort.country == mc].copy()
    dgs = mc_df.ICD_code.unique()
    
    for dg in dgs:
        mc_dg = mc_df.loc[mc_df.ICD_code==dg].sort_values("year", ascending=True).reset_index(drop=True)
        if len(mc_dg) > 0:
            
            imputed_row = dict({"ICD_code":dg, "country":mc})
            for y in range(1984, 2019):
                if y not in mc_dg.year.values:
                    count = count+1
                    imputed_row["year"] = y
                    right_after = mc_dg.loc[mc_dg.year > y]
                    if len(right_after) > 0:
                        imputed_row["deaths"] = mc_dg.iloc[0]["deaths"]
                    else:
                        right_before = mc_dg.loc[mc_dg.year < y]
                        imputed_row["deaths"] = right_before.iloc[-1]["deaths"]

                    mort_complete = mort_complete.append(imputed_row, ignore_index=True)



In [762]:
mort = mort_complete


In [763]:
mort["total_deaths"] = mort.groupby(["year", "ICD_code"])["deaths"].transform("sum")

In [764]:
mort["deaths"] = mort["deaths"].apply(float)
mort["total_deaths"] = mort["total_deaths"].apply(float)
mort["death_share"] = mort["deaths"] / mort["total_deaths"]

In [765]:
mort_agg  = mort.groupby(["year","ICD_code"])["deaths"].sum().reset_index()

In [766]:
mort_agg["total_deaths"] = mort_agg.groupby("year")["deaths"].transform("sum")
mort_agg["prevalence"] = mort_agg["deaths"] / mort_agg["total_deaths"]

In [767]:
mort_agg = mort_agg[["year", "ICD_code", "prevalence"]]

In [768]:
mort.to_csv("data/mortality_by_disease.csv", sep="\t", index=False)

## Get population data

In [382]:
pop = pd.read_csv("population.csv")

In [383]:
pop = pop.loc[~pop["Series Code"].isnull()]

In [384]:
pop = pop.drop(["Series Name", "Series Code"], axis=1)\
    .rename(columns={"Country Name":"country", "Country Code":"country_code"})

In [385]:
pop = pop.drop("2018 [YR2018]",axis=1)

In [386]:
pop_world = pop.loc[pop.country == "World"]
pop = pop.loc[:216]

In [387]:
pop_world = pd.melt(pop_world, id_vars=["country", "country_code"])
pop = pd.melt(pop, id_vars=["country", "country_code"])

In [388]:
pop_world = pop_world.rename(columns={"variable": "year", "value":"population"})
pop = pop.rename(columns={"variable": "year", "value":"population"})


In [389]:
pop_world["year"] = pop_world["year"].apply(lambda x: int(str(x)[0:4]))
pop["year"] = pop["year"].apply(lambda x: int(str(x)[0:4]))

In [390]:
pop.loc[pop.country=="Eswatini", "country"] = "Swaziland"


In [391]:
pop_world = pop_world.drop(["country", "country_code"], axis=1)
pop = pop.drop("country_code", axis=1)
pop_world = pop_world.rename(columns={"population":"world_population"})

In [394]:
pop = pop.loc[pop.population !=  ".."]

In [395]:
pop = pd.merge(left=pop, right=pop_world, on="year",how="left")
pop["population"] = pop["population"].apply(float)
pop["world_population"] = pop["world_population"].apply(float)

pop["population_share"] = pop["population"] / pop["world_population"]

In [493]:
pop = pop.sort_values(["country", "year"], ascending=True).reset_index(drop=True)

In [494]:
pop.to_csv("data/population.csv", sep="\t", index=False)

## Get Income Level data

In [307]:
income_df = pd.read_excel("income/OGHIST.xls", sheet_name="2002")

## get CL episodes

In [309]:
cl_ep = pd.read_excel("CL_episodes.xlsx")

In [310]:
cl_ep = cl_ep.loc[~cl_ep.disease_group.isnull()]

In [311]:
cl_ep = cl_ep[["year", "country", "disease_group", "outcome"]]

In [312]:
cl_ep.head()

,year,country,disease_group,outcome
0,2001,Brazil,"B20,B21,B22,B23,B24",discount
1,2001,Brazil,"B20,B21,B22,B23,B24",discount
2,2001,Brazil,"B20,B21,B22,B23,B24",discount
3,2001,USA,A01||A02||A04||A09||A20||A22||A54,discount
4,2001,Germany,"B20,B21,B22,B23,B24",CL


In [313]:
cl_ep_unstacked = pd.DataFrame(columns=["year", "country", "ICD_code", "outcome"])
for i, row in cl_ep.iterrows():
    new_row=dict()
    new_row["year"] = row["year"]
    new_row["country"] = row["country"]
    new_row["outcome"] = row["outcome"]
    dgs = row["disease_group"].split("||")
    for dg in dgs:
        new_row["ICD_code"] = dg
        cl_ep_unstacked = cl_ep_unstacked.append(new_row, ignore_index=True)
    

In [314]:
cl_ep_unstacked["CL"] = 0
cl_ep_unstacked.loc[cl_ep_unstacked.outcome=="CL","CL"] = 1

cl_ep_unstacked["discount"] = 0
cl_ep_unstacked.loc[cl_ep_unstacked.outcome=="discount","discount"] = 1

In [315]:
cl_ep_unstacked["request_cumulative"] = cl_ep_unstacked.groupby(["ICD_code"]).cumcount() + 1
cl_ep_unstacked["CL_cumulative"] = cl_ep_unstacked.groupby(["ICD_code"])["CL"].cumsum()
cl_ep_unstacked["discount_cumulative"] = cl_ep_unstacked.groupby(["ICD_code"])["discount"].cumsum()


In [316]:
cl_ep_unstacked.loc[cl_ep_unstacked.country=="USA", "country"] = "United States"
cl_ep_unstacked.loc[cl_ep_unstacked.country=="Eretria", "country"] = "Eritrea"

cl_ep_unstacked.to_csv("data/cl_ep_unstacked.csv", sep="\t", index=False)

In [319]:
cl_ep_cumulative = cl_ep_unstacked.groupby(["ICD_code","year"]).agg({"request_cumulative":"max",
                                                "CL_cumulative":"max",
                                                "discount_cumulative":"max"
                                               }).reset_index()

In [320]:
def find_cum_country(row, eps, outcome):
    search_code = row["ICD_code"]
    search_year = row["year"]
    # all countries that have requested so far
    if outcome == "request":
        countries = eps.loc[(eps.ICD_code == search_code) & (eps.year <= search_year)].country.unique()
    # all countries that have issued CLs so far
    if outcome == "CL":
        countries = eps.loc[(eps.ICD_code == search_code) & (eps.year <= search_year) & 
                           (eps.outcome == "CL")].country.unique()
    if outcome == "discount":
    # all countries that have issued discounts so far
        countries = eps.loc[(eps.ICD_code == search_code) & (eps.year <= search_year) & 
                           (eps.outcome == "discount")].country.unique()
    
    countries.sort()
    return countries

In [321]:
cl_ep_cumulative["request_country"] = cl_ep_cumulative.apply(lambda x: find_cum_country(x,cl_ep_unstacked, "request"),
                                                             axis=1)

In [322]:
cl_ep_cumulative["CL_country"] = cl_ep_cumulative.apply(lambda x: find_cum_country(x,cl_ep_unstacked, "CL"),
                                                             axis=1)

In [323]:
cl_ep_cumulative["discount_country"] = cl_ep_cumulative.apply(lambda x: find_cum_country(x,cl_ep_unstacked, "discount"),
                                                             axis=1)

In [325]:
cl_ep_cumulative.to_csv("data/cl_ep_cumulative.csv", sep='\t', index=False)

In [326]:
# if there is a missing year after the first issuance of CL, replicate the previous row
cl_ep_cumulative_complete = pd.DataFrame(columns=cl_ep_cumulative.columns)
codes = cl_ep_cumulative.ICD_code.unique()

for c in codes:
    c_cum = cl_ep_cumulative.loc[cl_ep_cumulative.ICD_code == c].copy()
    year = c_cum.year.min()
    for y in range(year, 2019):
        if y in c_cum.year.values:
            row=c_cum.loc[c_cum.year==y].copy()
        else:
            row["year"] = y
            
        cl_ep_cumulative_complete = cl_ep_cumulative_complete.append(row, ignore_index=True)
    

In [327]:
cl_ep_cumulative_complete

,ICD_code,year,request_cumulative,CL_cumulative,discount_cumulative,request_country,CL_country,discount_country
0,A01,2001,2,0,2,"[Canada, United States]",[],"[Canada, United States]"
1,A01,2002,2,0,2,"[Canada, United States]",[],"[Canada, United States]"
2,A01,2003,2,0,2,"[Canada, United States]",[],"[Canada, United States]"
3,A01,2004,2,0,2,"[Canada, United States]",[],"[Canada, United States]"
4,A01,2005,2,0,2,"[Canada, United States]",[],"[Canada, United States]"
5,A01,2006,2,0,2,"[Canada, United States]",[],"[Canada, United States]"
6,A01,2007,2,0,2,"[Canada, United States]",[],"[Canada, United States]"
7,A01,2008,2,0,2,"[Canada, United States]",[],"[Canada, United States]"
8,A01,2009,2,0,2,"[Canada, United States]",[],"[Canada, United States]"
9,A01,2010,2,0,2,"[Canada, United States]",[],"[Canada, United States]"


In [328]:
cl_ep_cumulative_complete.to_csv("data/cl_ep_cumulative_complete.csv", sep='\t', index=False)

In [329]:
by_disease_group_complete.head()

,ICD_code,custom_group,neglected,filing_year,patent_count,log_patent_count0,log_patent_count1
0,A00,Cholera,True,1984,4,1.386294,1.609438
1,A00,Cholera,True,1985,3,1.098612,1.386294
2,A00,Cholera,True,1986,7,1.945910,2.079442
3,A00,Cholera,True,1987,8,2.079442,2.197225
4,A00,Cholera,True,1988,9,2.197225,2.302585


In [820]:
cl_ep_cumulative_complete

,ICD_code,year,request_cumulative,CL_cumulative,discount_cumulative,request_country,CL_country,discount_country
0,A01,2001,2,0,2,"[Canada, United States]",[],"[Canada, United States]"
1,A01,2002,2,0,2,"[Canada, United States]",[],"[Canada, United States]"
2,A01,2003,2,0,2,"[Canada, United States]",[],"[Canada, United States]"
3,A01,2004,2,0,2,"[Canada, United States]",[],"[Canada, United States]"
4,A01,2005,2,0,2,"[Canada, United States]",[],"[Canada, United States]"
5,A01,2006,2,0,2,"[Canada, United States]",[],"[Canada, United States]"
6,A01,2007,2,0,2,"[Canada, United States]",[],"[Canada, United States]"
7,A01,2008,2,0,2,"[Canada, United States]",[],"[Canada, United States]"
8,A01,2009,2,0,2,"[Canada, United States]",[],"[Canada, United States]"
9,A01,2010,2,0,2,"[Canada, United States]",[],"[Canada, United States]"


# combine patent data with CL data

In [839]:
patent_CL_df = pd.merge(left = by_disease_group_complete, right=cl_ep_cumulative_complete, 
                        left_on=["ICD_code", "filing_year"], right_on=["ICD_code", "year"], how="left"
                        ).drop("year", axis=1)

In [840]:
patent_CL_df[["request_cumulative", "CL_cumulative", "discount_cumulative"]] = patent_CL_df[["request_cumulative", "CL_cumulative", "discount_cumulative"]].fillna(0)

In [841]:
request_treated = cl_ep_cumulative_complete.loc[cl_ep_cumulative_complete.request_cumulative > 0].ICD_code.unique()
CL_treated = cl_ep_cumulative_complete.loc[cl_ep_cumulative_complete.CL_cumulative > 0].ICD_code.unique()
discount_treated = cl_ep_cumulative_complete.loc[cl_ep_cumulative_complete.discount_cumulative > 0].ICD_code.unique()
patent_CL_df["request_treated"] = 0
patent_CL_df["CL_treated"] = 0
patent_CL_df["discount_treated"] = 0
patent_CL_df.loc[patent_CL_df.ICD_code.isin(request_treated), "request_treated"] = 1
patent_CL_df.loc[patent_CL_df.ICD_code.isin(CL_treated), "CL_treated"] = 1
patent_CL_df.loc[patent_CL_df.ICD_code.isin(discount_treated), "discount_treated"] = 1


In [842]:
patent_CL_df = patent_CL_df.reset_index()
patent_CL_df = patent_CL_df.rename(columns={"index":"orig_index"})

In [843]:
def impute_market_data(row, field):
    search_code = row["ICD_code"]
    search_year = row["year"]
    search_country = row["country"]
    
    #outcomes
    imputed = np.nan
    
    if (field == "deaths") | (field=="death_share"): 
        match_mort = mort.loc[(mort.ICD_code == search_code) & (mort.country == search_country)].copy()

        # in mortality data, search first row after or last row before
        if len(match_mort) > 0:
            print(search_code, search_year, search_country)
            match_mort = match_mort.sort_values("year", ascending=True).reset_index(drop=True)

            mort_right_after = match_mort.loc[match_mort.year > search_year]
            if len(mort_right_after) > 0: 
                imputed = mort_right_after.iloc[0][field]

            mort_right_before = match_mort.loc[match_mort.year < search_year]
            if len(mort_right_before) > 0:
                imputed = mort_right_before.iloc[-1][field]
    
    if (field == "population_share"):
        match_pop = pop.loc[(pop.country == search_country)].copy()
    
        if len(match_pop) > 0:
            match_pop = match_pop.sort_values("year", ascending=True).reset_index(drop=True)

            pop_right_after = match_pop.loc[match_pop.year > search_year]
            if len(pop_right_after) > 0:
                imputed = pop_right_after.iloc[0][field]

            pop_right_before = match_pop.loc[match_pop.year < search_year]
            if len(pop_right_before) > 0:
                imputed = pop_right_before.iloc[-1][field]
    
    return imputed


In [844]:
def get_country_weight(country_field):
    country_weight = pd.DataFrame(columns=["orig_index", "ICD_code", "year", "country"])
    for i, row in patent_CL_df.iterrows():
        new_row=dict()
        new_row["ICD_code"] = row["ICD_code"]
        new_row["year"] = row["filing_year"]
        new_row["orig_index"] = row["orig_index"]
        countries = row[country_field]
        if isinstance(countries, float):
            continue
        if len(countries) == 0:
            continue
        if isinstance(countries, str):
            countries = ast.literal_eval(countries)
        
        for c in countries:
            new_row["country"] = c
            country_weight = country_weight.append(new_row, ignore_index=True)
            
    # add mortality weight
    country_weight = pd.merge(left = country_weight, right=mort[["year", "ICD_code", "country", "deaths", "death_share"]], 
                            on=["year", "ICD_code", "country"], how="left")
    # add population weight 
    country_weight = pd.merge(left = country_weight, right=pop[["year", "country", "population_share"]],
                          on=["year", "country"], how="left")
    
    # add income cataegory 
    country_weight = pd.merge(left = country_weight, right = income_df[["country", "income_category"]], 
                              on="country", how="left")

    income_dummies = pd.get_dummies(country_weight.income_category)
    country_weight = pd.concat([country_weight, income_dummies], axis=1)
    #impute missing data
    country_weight.loc[country_weight.deaths.isnull(), "deaths"] = \
        country_weight.loc[country_weight.deaths.isnull()].apply(lambda x: impute_market_data(x, field="deaths") ,
                                                                 axis=1)
    country_weight.loc[country_weight.death_share.isnull(), "death_share"] = \
        country_weight.loc[country_weight.death_share.isnull()].apply(lambda x: impute_market_data(x, field="death_share") ,
                                                                 axis=1)
    country_weight.loc[country_weight.population_share.isnull(), "population_share"] = \
        country_weight.loc[country_weight.population_share.isnull()].apply(lambda x: impute_market_data(x, field="population_share") ,
                                                                 axis=1)

    country_weight.loc[country_weight.deaths.isnull(), "deaths"] = 0
    country_weight.loc[country_weight.death_share.isnull(), "death_share"] = 0
    country_weight.loc[country_weight.population_share.isnull(), "population_share"] = 0


    weight_dict=dict()
    weight_dict["deaths"] = "sum"
    weight_dict["death_share"] = "sum"
    weight_dict["population_share"] = "sum"
    for ic in country_weight.income_category.unique():
        death_interacted = "deaths" + "_" + ic
        country_weight[death_interacted] = country_weight["deaths"] *country_weight[ic]
        weight_dict[death_interacted] = "sum"
        
        death_share_interacted = "death_share" + "_" + ic
        country_weight[death_share_interacted] = country_weight["death_share"] *country_weight[ic]
        weight_dict[death_share_interacted] = "sum"
        
        population_share_interacted = "population_share" + "_" + ic
        country_weight[population_share_interacted] = country_weight["population_share"] *country_weight[ic]
        weight_dict[population_share_interacted] = "sum"

        
    country_weight = country_weight.groupby("orig_index").agg(weight_dict).reset_index()
    
    col_label = country_field[:-7]
    new_names = dict()
    for name in country_weight.columns[1::]:
        new_names[name] = col_label+name
    country_weight = country_weight.rename(columns=new_names)
    
    
    return country_weight

In [845]:
request_weight = get_country_weight("request_country")
CL_weight = get_country_weight("CL_country")
discount_weight = get_country_weight("discount_country")
patent_CL_df = pd.merge(left = patent_CL_df, right = request_weight, 
                        on="orig_index", how="left")
patent_CL_df = pd.merge(left = patent_CL_df, right = CL_weight, 
                        on="orig_index", how="left")
patent_CL_df = pd.merge(left = patent_CL_df, right = discount_weight, 
                        on="orig_index", how="left")


In [847]:
patent_CL_df = patent_CL_df.drop(["orig_index","request_country", "CL_country", "discount_country"], axis=1)
patent_CL_df.loc[:, (patent_CL_df.columns != 'log_patent_count0')] = patent_CL_df.loc[:, (patent_CL_df.columns != 'log_patent_count0')].fillna(0)


In [819]:
patent_CL_df.loc[patent_CL_df.request_treated == 1].disease_code.unique()

array(['A01', 'A02', 'A04', 'A09', 'A20', 'A22', 'A54',
       'B15,B16,B17,B18,B19', 'B20,B21,B22,B23,B24'], dtype=object)

In [818]:
patent_CL_df.loc[(patent_CL_df.request_treated == 1) & (patent_CL_df.neglected ==1)].disease_code.unique()

array(['A01', 'A02', 'B20,B21,B22,B23,B24'], dtype=object)

In [815]:
patent_CL_df.loc[ (patent_CL_df.neglected ==1)].disease_code.unique()

array(['A00', 'A01', 'A02', 'A03', 'A06,A07', 'A15,A16', 'A17,A18,A19',
       'A30', 'A31', 'A66', 'A70,A71,A74', 'A82', 'A92', 'A93,A94',
       'A96,A98,A99', 'A97', 'B20,B21,B22,B23,B24',
       'B35,B36,B37,B38,B39,B40,B41,B42,B43,B44,B45,B46,B47,B48,B49',
       'B50,B51,B52,B53,B54', 'B55', 'B56,B57', 'B65', 'B67', 'B68',
       'B72,B73,B74,B75,B76,B77,B78,B79,B80', 'B81,B82,B83', 'B86'],
      dtype=object)

## add overall disease prevalence for each year

In [856]:
patent_CL_df = pd.merge(left=patent_CL_df, right=mort_agg, left_on=["ICD_code", "filing_year"],
                        right_on=["ICD_code", "year"], how="left")

In [857]:
patent_CL_df.loc[patent_CL_df.prevalence.isnull(), "prevalence"] = 0
patent_CL_df = patent_CL_df.drop("year", axis=1)


In [858]:
patent_CL_df["min_year"] = patent_CL_df.groupby("ICD_code")["filing_year"].transform("min")

In [859]:
patent_CL_df = patent_CL_df.rename(columns={"ICD_code":"disease_code", "custom_group":"disease_group",
                                           "filing_year":"year"})

In [860]:
patent_CL_df["neglected"] = patent_CL_df["neglected"].apply(int)

In [863]:
patent_CL_df.to_csv("data/patent_CL_df.csv", sep='\t', index=False)

In [899]:
patent_CL_df.discount_population_share_L.describe()

AttributeError: 'DataFrame' object has no attribute 'discount_population_share_L'

In [864]:
patent_CL_df.columns

Index(['disease_code', 'disease_group', 'neglected', 'year', 'patent_count',
       'log_patent_count0', 'log_patent_count1', 'request_cumulative',
       'CL_cumulative', 'discount_cumulative', 'request_treated', 'CL_treated',
       'discount_treated', 'request_deaths', 'request_death_share',
       'request_population_share', 'request_deaths_H', 'request_death_share_H',
       'request_population_share_H', 'request_deaths_LM',
       'request_death_share_LM', 'request_population_share_LM',
       'request_deaths_UM', 'request_death_share_UM',
       'request_population_share_UM', 'request_deaths_L',
       'request_death_share_L', 'request_population_share_L', 'CL_deaths',
       'CL_death_share', 'CL_population_share', 'CL_deaths_H',
       'CL_death_share_H', 'CL_population_share_H', 'CL_deaths_LM',
       'CL_death_share_LM', 'CL_population_share_LM', 'CL_deaths_UM',
       'CL_death_share_UM', 'CL_population_share_UM', 'CL_deaths_L',
       'CL_death_share_L', 'CL_population_sha

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,10))
dg="B20,B21,B22,B23,B24"
dg_label = by_disease_group.loc[by_disease_group.ICD_code==dg].custom_group.unique()
dg_patent = by_disease_group.loc[by_disease_group.ICD_code==dg]
dg_patent = dg_patent.loc[(dg_patent.publication_year >= 1990)]
x=dg_patent.publication_year
y=dg_patent.log_patent_count
plt.plot(x,y, label=dg_label)
plt.axvline(x=1995, color="red", label="TRIPS")
plt.axvline(x=2002, color="green", label="Doha")
CL = disease_group.loc[disease_group.custom_group_code == dg]["CL_year"].unique()
if CL is not None:
    plt.axvline(x=CL, color="orange", label="CL")

plt.legend()

In [ ]:
CL

In [ ]:
hiv_patents = matched_patents_fam.loc[matched_patents_fam.ICD_code== "B20,B21,B22,B23,B24"]

In [ ]:
hiv_ipcs = []
for ipc in hiv_patents.ipc_codes:
    if isinstance(ipc, str):
        hiv_ipcs = hiv_ipcs + ipc.split(",")

In [ ]:
hiv_assignees = []
for ah in hiv_patents.assignees_harmonized:
    if isinstance(ah, str):
        hiv_assignees = hiv_assignees + ah.split("||")

In [ ]:
len(set(hiv_assignees))

In [ ]:
import collections

In [ ]:
ipc_freq = pd.DataFrame.from_dict(dict(collections.Counter(hiv_ipcs)), orient="index").reset_index()
ipc_freq.columns = ["subclass", "cnt"]
ipc_freq = ipc_freq.sort_values("cnt", ascending=False)
ipc_freq.loc[ipc_freq.cnt > 100]

In [ ]:
assignee_freq = pd.DataFrame.from_dict(dict(collections.Counter(hiv_assignees)), orient="index").reset_index()
assignee_freq.columns = ["assignee", "cnt"]
assignee_freq = assignee_freq.sort_values("assignee", ascending=False)
#ipc_freq.loc[ipc_freq.cnt > 100]

In [ ]:
assignee_freq.loc[assignee_freq.cnt > 50]

In [ ]:
hiv_patents.shape

In [ ]:
drugbank_hiv = pd.read_csv("drugbank_hiv_list.csv", sep='\t')

In [ ]:
hiv_CL = pd.DataFrame(columns=hiv_patents.columns)
for drugbank_patent in drugbank_hiv.publication_number:
    hiv_CL = hiv_CL.append(hiv_patents[hiv_patents.publication_number.str.contains(drugbank_patent)])

In [ ]:
hiv_ipcs_CL = []
for ipc in hiv_CL.ipc_codes:
    if isinstance(ipc, str):
        hiv_ipcs_CL = hiv_ipcs_CL + ipc.split(",")


In [ ]:
hiv_ipcs_CL = list(set(hiv_ipcs_CL))

In [ ]:
#hiv_patents_stacked = pd.DataFrame(columns=["publication_number","publication_year","subclass"])
#hiv_patents = hiv_patents.reset_index(drop=True)
for i, row in hiv_patents.iloc[16453:].iterrows(): 
    ipcs = row["ipc_codes"].split(",")
    new_row = dict()
    new_row["publication_number"] = row["publication_number"]
    #new_row["assignees"] = row["assignees_harmonized"]
    new_row["publication_year"] = row["publication_year"]
    for i in ipcs:
        new_row["subclass"] = i
        hiv_patents_stacked = hiv_patents_stacked.append(new_row, ignore_index=True)

In [ ]:
i

In [ ]:
hiv_patents.shape

In [ ]:
hiv_patents_stacked

In [ ]:
hiv_patents_stacked["publication_year"] = hiv_patents_stacked["publication_year"].apply(int)
by_hiv_subclass = hiv_patents_stacked.loc[hiv_patents_stacked.publication_year >= 1985]
by_hiv_subclass = by_hiv_subclass.loc[by_hiv_subclass.publication_year <= 2016]

by_hiv_subclass = by_hiv_subclass.groupby(["subclass", "publication_year"])["publication_number"].count().reset_index()

In [ ]:
major_subclass = by_hiv_subclass.groupby("subclass").publication_number.mean().reset_index()
major_subclass = major_subclass.loc[major_subclass.publication_number >= 10].subclass.unique()

In [ ]:
by_hiv_subclass = by_hiv_subclass.loc[by_hiv_subclass.subclass.isin(major_subclass)]

In [ ]:
by_hiv_subclass["CL"] = "No CL"
by_hiv_subclass.loc[by_hiv_subclass.subclass.isin(hiv_ipcs_CL),"CL"] = "CL"

In [ ]:
by_hiv_subclass["pre2002"] = "pre"
by_hiv_subclass.loc[by_hiv_subclass.publication_year >=2002, "pre2002"] = "post"

In [ ]:
by_hiv_subclass.groupby(["CL","pre2002"]).agg({"publication_number":"mean"}).reset_index()

In [ ]:
by_hiv_subclass.subclass.nunique()

In [ ]:
by_hiv_subclass.loc[by_hiv_subclass.CL=="No CL"].subclass.nunique()

In [ ]:
by_hiv_subclass_summary = by_hiv_subclass.groupby("subclass").\
    agg({"publication_year":"min", "publication_number":"mean"}).reset_index()
by_hiv_subclass_summary.columns = ["subclass", "first_publication_year", "patent_cnt"]
by_hiv_subclass_summary["CL"] = False
by_hiv_subclass_summary.loc[by_hiv_subclass_summary.subclass.isin(hiv_ipcs_CL),"CL"] = True

In [ ]:
by_hiv_subclass_summary.CL.unique()

In [ ]:
by_hiv_subclass

In [ ]:
by_hiv_subclass_summary

In [ ]:
matched_patents_full.groupby("country_code").publication_number.count()
